In [ ]:
import sys 
sys.path.append('../../')

In [2]:
import os
from openai import OpenAI

openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

user_prompt_template_m_turns = """
[Instruction]\nPlease act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. 
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response.
Begin your evaluation by providing a short explanation. Be as objective as possible.
After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: \"[[rating]]\", 
for example: \"Rating: [[5]]\".\n\n
There may be multiple turns of conversations between user and AI assistant.
[Conversation]\n{content}\n[The End of Conversation]\n
"""

def llm_reviewer_m_turns(content):
    try:
        # Format the user_prompt_template with the question and answer
        formatted_prompt = user_prompt_template_m_turns.format(content=content)
        
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": formatted_prompt}
            ],
            temperature = 0.0,
            seed = 42,
            max_tokens = 1024,
            #logprobs = True
        )
        questions = response.choices[0].message.content
        return questions
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



In [10]:
import json

# Assuming the file 'task_1.jsonl' is in the same directory as this script
file_path = 'task_1.jsonl'

# Read the file and process each line
conversation_string = ''
with open(file_path, 'r') as file:
    for line in file:
        obj = json.loads(line)
        conversation_string += obj['role'] + ": " + obj['content'] + '\n'

print(conversation_string)

User: Could you help me refactor this greedy algorithm that's supposed to find the minimum number of platforms needed at a railway station to avoid any delay in arrival of any train? I'd like it to also return the schedule of trains (as pairs of arrival and departure times) for each platform. Here's the current code: ```python def findPlatform(arrival, departure): arrival.sort() departure.sort() plat_needed = 1 result = 1 i = 1 j = 0 while i < len(arrival) and j < len(departure): if arrival[i] <= departure[j]: plat_needed += 1 i += 1 else: plat_needed -= 1 j += 1 if plat_needed > result: result = plat_needed return result ```
Assistant: In order to achieved what you have asked, you will need to use a much more complex or detailed code so that we can track which train goes to which platform properly.

Here is good way to go about this:
Assistant: def findPlatform(arrival, departure):
    events = sorted([(t, 'a') for t in arrival] + [(t, 'd') for t in departure])

    plat_needed = 0
  

In [11]:
rating = llm_reviewer_m_turns(conversation_string)
if rating:
    print(rating)

The AI assistant's response is helpful and relevant to the user's request. It provides a refactored version of the algorithm that not only finds the minimum number of platforms needed but also returns the schedule of trains for each platform. The code is accurate and demonstrates a good understanding of the problem by sorting the events (both arrivals and departures) and iterating through them to allocate platforms and schedule trains accordingly.

The assistant's solution uses a list of tuples to track events, a smart way to handle the arrivals and departures in a single sorted list. The code also includes comments that help explain the logic, which is a good practice for code readability and maintenance.

However, there are a few points that could be improved or clarified:
1. The code assumes that each train's arrival and departure times are unique, which may not always be the case in real-world scenarios.
2. The code does not handle edge cases where a departure time could be earlier

In [3]:
import os
import json

directory_path = 'batch_2_results'
ratings = []

# Open a text file to write the ratings
with open('llm_reviews.txt', 'w') as record_file:
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(directory_path, filename)
            conversation_string = ''
            
            # Read the file and process each line
            with open(file_path, 'r') as file:
                for line in file:
                    obj = json.loads(line)
                    conversation_string += obj['role'] + ": " + obj['content'] + '\n'
            
            # Get the rating for the current file
            rating = llm_reviewer_m_turns(conversation_string)
            if rating:
                ratings.append((filename, rating))
                # Write the rating into the text file
                record_file.write(f"{filename}: {rating}\n")